# Model Selcection 모듈

## - train/ test 데이터를 분리하지 않고 머신러닝 수행

In [2]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)

DecisionTreeClassifier()

In [4]:
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target,pred) # 1.0 #;즉 과적합 발생함

1.0

# cross_validate 메쏘드

In [8]:
from sklearn.model_selection import cross_validate

In [10]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0.        , 0.00100112, 0.00099397, 0.00099754, 0.0009973 ]),
 'score_time': array([0.00099754, 0.        , 0.        , 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])}

In [12]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target,return_train_score = True)

{'fit_time': array([0.00199509, 0.        , 0.0009973 , 0.00099683, 0.        ]),
 'score_time': array([0.       , 0.       , 0.       , 0.       , 0.0009985]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

## * Train/Test 데이터 셋을 분리

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(  # Notation: 1차원 데이터는 소문자, 2차원이상 데이터부터 대분자로 표기
 iris.data, iris.target, test_size=0.2, random_state=2021  # 독립변수데이터, 종속변수 데이터
)

In [17]:
import pandas as pd
pd.Series(y_test).value_counts()

0    14
1    10
2     6
dtype: int64

## * Stratified 분리
###  stratified k 폴드 : train_test split할때 stratify 속성을 주어 골고루 뽑히도록 한다.

In [25]:
X_train, X_test, y_train, y_test = train_test_split(  # Notation: 1차원 데이터는 소문자, 2차원이상 데이터부터 대분자로 표기
iris.data, iris.target, test_size=0.2, random_state=2021, # 독립변수데이터, 종속변수 데이터
stratify = iris.target #골고루 뽑히도록 속성 추가
) 
pd.Series(y_test).value_counts()

0    10
1    10
2    10
dtype: int64

## * cross_val_score() method

In [31]:
from sklearn.model_selection import  cross_val_score

In [32]:
# 성능 지표는 정확도(accuracy), 교차검증세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 0.96666667, 1.        ])

In [34]:
import numpy as np
scores = cross_val_score(dtc, iris.data, iris.target, cv =5)
np.mean(scores)

0.9533333333333334

# GridSearchCV
### 교차 검증과 최적 하이퍼 파라메터 튜닝을 한꺼번에 수행
 - 하이퍼 파라미터 : classifier나 regressor에 사용되는 파라미터, 예측성능개선에 도움
 - GridSearchCV의 파라미터 : estimator: classifier, regressor, pipeline

In [36]:
dtc = DecisionTreeClassifier(random_state=2021)

In [39]:
# parameter를 딕셔너리 형대로 설정
params = {
    'max_depth' : [2,3,4,5],
    'min_samples_split' : [2,3]
}

In [42]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid= params, cv = 3) #교차검증의 개수(cv)가 3개 

In [44]:
grid_dtc.fit(X_train,y_train) # 교차검정, 하이퍼파라메터 두번함

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [45]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00033236, 0.00099715, 0.00099746, 0.00066471, 0.00066153,
        0.00099683, 0.00033116, 0.00099762]),
 'std_fit_time': array([4.70021655e-04, 5.61957980e-07, 8.14880438e-04, 4.70022017e-04,
        4.67779656e-04, 1.21570099e-06, 4.68335781e-04, 8.10467325e-07]),
 'mean_score_time': array([0.00066439, 0.        , 0.00033283, 0.        , 0.00033291,
        0.00033434, 0.00033236, 0.        ]),
 'std_score_time': array([0.0004698 , 0.        , 0.0004707 , 0.        , 0.00047081,
        0.00047283, 0.00047002, 0.        ]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split'

In [47]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [49]:
# 최고정확도
grid_dtc.best_score_

0.9583333333333334

In [50]:
# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test,y_test)

0.9